In [38]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from flygym import Fly, Camera, SingleFlySimulation
from flygym.examples import PreprogrammedSteps

from flygym.arena.tethered import Tethered

In [39]:
timestep = 1e-4

# Define leg raise correction vectors
correction_vectors = {
    # "leg pos": (Coxa, Coxa_roll, Coxa_yaw, Femur, Femur_roll, Tibia, Tarsus1)
    # unit: radian
    "F": np.array([-0.03, 0, 0, -0.03, 0, 0.03, 0.03]),
    "M": np.array([-0.015, 0.001, 0.025, -0.02, 0, -0.02, 0.0]),
    "H": np.array([0, 0, 0, -0.02, 0, 0.01, -0.02]),
}

"""correction_vectors = {
    # "leg pos": (Coxa, Coxa_roll, Coxa_yaw, Femur, Femur_roll, Tibia, Tarsus1)
    # unit: radian
    "F": np.array([-0.03, 0, 0, -0.04, 0, 0.032, 0.02]),
    "M": np.array([-0.015, 0.001, 0.2, 0.004, 0, 0.01, -0.008]),
    "H": np.array([0, 0, 0, -0.01, 0, 0.005, 0]),
}"""

right_leg_inversion = [1, -1, -1, 1, -1, 1, 1]

# Define leg raise rates
correction_rates = {"retraction": (500, 400), "stumbling": (2000, 1900)}
max_increment = 80

In [40]:
fly = Fly(
    enable_adhesion=True,
    draw_adhesion=True,
    init_pose="tripod",
    control="position",
    spawn_pos=np.concatenate([np.random.rand(2), [0]]) - np.concatenate([np.ones(2)*0.5, [0]]),
    detect_flip=True,
)

cam = Camera(fly=fly, play_speed=0.05, camera_id="Animat/camera_right")
cam.cam_offset += [0.0, 4.0, 0.0]
arena = Tethered()
sim = SingleFlySimulation(
    fly=fly,
    cameras=[cam],
    timestep=1e-4,
    arena=arena,
)

cam.update_camera_pos = False

obs, _ = sim.reset()

In [41]:
prepro_step = PreprogrammedSteps()
all_legs = prepro_step.legs

leg_joints_ids = {leg: [i for i, jnt in enumerate(fly.actuated_joints) if jnt.startswith("joint_" + leg)] for leg in all_legs}

default_pose =np.zeros(len(fly.actuated_joints))
for leg in all_legs:
    default_pose[leg_joints_ids[leg]] = prepro_step.get_joint_angles(leg, 0.0, 1.0)

leg_joints_ids, default_pose

({'LF': [0, 1, 2, 3, 4, 5, 6],
  'LM': [7, 8, 9, 10, 11, 12, 13],
  'LH': [14, 15, 16, 17, 18, 19, 20],
  'RF': [21, 22, 23, 24, 25, 26, 27],
  'RM': [28, 29, 30, 31, 32, 33, 34],
  'RH': [35, 36, 37, 38, 39, 40, 41]},
 array([ 0.6101972 ,  1.32109394, -0.11010561, -2.60880835,  0.91672976,
         2.1007223 , -0.41476448,  0.40218574,  1.8716873 ,  0.80052746,
        -1.4432551 , -0.19805689,  1.98889966, -0.57303714,  0.49221796,
         2.49478087,  0.42962744, -1.52780084, -0.41317017,  1.39048667,
         0.04873025,  0.6101972 , -1.32109394,  0.11010561, -2.60880835,
        -0.91672976,  2.1007223 , -0.41476448,  0.40218574, -1.8716873 ,
        -0.80052746, -1.4432551 ,  0.19805689,  1.98889966, -0.57303714,
         0.49221796, -2.49478087, -0.42962744, -1.52780084,  0.41317017,
         1.39048667,  0.04873025]))

In [42]:
num_steps_per_leg = 1000
correction = 0.0

for i, leg in enumerate(["RF", "RM", "RH"]):
    correction = 0.0
    for j in trange(num_steps_per_leg):
        if j < num_steps_per_leg // 2:
            correction += correction_rates["retraction"][0] * sim.timestep
        else:
            correction -= correction_rates["retraction"][1] * sim.timestep

        correction = np.clip(correction, -1*max_increment, max_increment)
        joint_angles_corr = correction * correction_vectors[leg[1]]
        if leg[0] == "R":
            joint_angles_corr *= right_leg_inversion
        
        joint_angles = default_pose.copy()
        joint_angles[leg_joints_ids[leg]] += joint_angles_corr

        action = {"joints": joint_angles, "adhesion": np.zeros(6)}  
        obs, _, _, _, _ = sim.step(action)
        sim.render()

cam.save_video("outputs/stumbling_right.mp4", 0)

100%|██████████| 1000/1000 [00:01<00:00, 599.59it/s]


In [43]:
fly = Fly(
    enable_adhesion=True,
    draw_adhesion=True,
    init_pose="tripod",
    control="position",
    spawn_pos=np.concatenate([np.random.rand(2), [0]]) - np.concatenate([np.ones(2)*0.5, [0]]),
    detect_flip=True,
)

cam = Camera(fly=fly, play_speed=0.05, camera_id="Animat/camera_front")
cam.cam_offset -= [4.0, 0.0, 0.0]
arena = Tethered()
sim = SingleFlySimulation(
    fly=fly,
    cameras=[cam],
    timestep=1e-4,
    arena=arena,
)

cam.update_camera_pos = False

obs, _ = sim.reset()

for i, leg in enumerate(all_legs):
    correction = 0.0
    for j in trange(num_steps_per_leg):
        if j < num_steps_per_leg // 2:
            correction += correction_rates["retraction"][0] * sim.timestep
        else:
            correction -= correction_rates["retraction"][1] * sim.timestep

        correction = np.clip(correction, -1*max_increment, max_increment)
        joint_angles_corr = correction * correction_vectors[leg[1]]
        if leg[0] == "R":
            joint_angles_corr *= right_leg_inversion
        
        joint_angles = default_pose.copy()
        joint_angles[leg_joints_ids[leg]] += joint_angles_corr

        action = {"joints": joint_angles, "adhesion": np.zeros(6)}  
        obs, _, _, _, _ = sim.step(action)
        sim.render()

cam.save_video("outputs/stumbling_front.mp4", 0)

100%|██████████| 1000/1000 [00:01<00:00, 572.39it/s]
